In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:,.3f}'.format

# ADNIMERGE and MRI

In [2]:
adnimerge = pd.read_csv('Files/ADNIMERGE_06Feb2025.csv')
adnimerge.head()

C:\Users\anisr\AppData\Local\Temp\ipykernel_16836\1460029489.py:1: DtypeWarning: Columns (19,20,21,50,51,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  adnimerge = pd.read_csv('Files/ADNIMERGE_06Feb2025.csv')


,RID,COLPROT,ORIGPROT,PTID,SITE,VISCODE,EXAMDATE,DX_bl,AGE,PTGENDER,...,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,FBB_bl,Years_bl,Month_bl,Month,M,update_stamp
0,2,ADNI1,ADNI1,011_S_0002,11,bl,2005-09-08,CN,74.300,Male,...,NaN,1.336,NaN,NaN,NaN,0.000,0.000,0,0,2023-07-07 04:59:40.0
1,3,ADNI1,ADNI1,011_S_0003,11,bl,2005-09-12,AD,81.300,Male,...,22.830,1.109,NaN,NaN,NaN,0.000,0.000,0,0,2023-07-07 04:59:40.0
2,3,ADNI1,ADNI1,011_S_0003,11,m06,2006-03-13,AD,81.300,Male,...,22.830,1.109,NaN,NaN,NaN,0.498,5.967,6,6,2023-07-07 04:59:40.0
3,3,ADNI1,ADNI1,011_S_0003,11,m12,2006-09-12,AD,81.300,Male,...,22.830,1.109,NaN,NaN,NaN,0.999,11.967,12,12,2023-07-07 04:59:40.0
4,3,ADNI1,ADNI1,011_S_0003,11,m24,2007-09-12,AD,81.300,Male,...,22.830,1.109,NaN,NaN,NaN,1.999,23.934,24,24,2023-07-07 04:59:40.0


In [3]:
adnimerge['Adjusted_Age'] = adnimerge['AGE'] + adnimerge['Years_bl']

In [4]:
df = adnimerge[['RID', 'VISCODE', 'DX', 'DX_bl','PTGENDER','PTEDUCAT','PTETHCAT','PTRACCAT','PTMARRY','APOE4','Adjusted_Age']]

In [5]:
mri = pd.read_csv("Imp_Files/UCSFFSX_mri_all.csv")
regions_dict = pd.read_csv("Files/mri_regional_dict_proper_all.csv")
mapping_dict = dict(zip(regions_dict['ADNI Code'], regions_dict['Region']))

In [6]:
mri.rename(columns=mapping_dict,inplace=True)

In [7]:
mri = mri.loc[:, ~mri.columns.duplicated()]

In [8]:
mri = mri[mri['STATUS'] == 'complete']
columns_to_keep = ['VISCODE2', 'RID'] + list(mapping_dict.values())
mri = mri[columns_to_keep]

In [9]:
mri_sel_lt = ['RightEntorhinal', 'RightTemporalPole', 'RightParahippocampal',
              'RightInferiorTemporal', 'RightMiddleTemporal', 'RightFusiform',
              'RightInferiorParietal', 'RightIsthmusCingulate', 'RightBankssts',
              'RightPrecuneus', 'RightHippocampus', 'RightAmygdala',
              'RightAccumbensArea', 'RightMedialOrbitofrontal', 'RightPallidum',
              'RightCaudalMiddleFrontal', 'RightPutamen',
              'RightRostralAnteriorCingulate', 'RightParacentral', 'RightPrecentral',
              'RightLingual', 'LeftEntorhinal', 'LeftHippocampus', 'LeftFusiform',
              'LeftMiddleTemporal', 'LeftParacentral',
              'RightInferiorLateralVentricle', 'LeftInferiorLateralVentricle',
       'RightLateralVentricle', 'LeftLateralVentricle',  'Icv']

In [10]:
viscode_counts = mri.groupby('RID')['VISCODE2'].count()
sorted_viscode_counts = viscode_counts.sort_values(ascending=False)

In [11]:
rids_with_viscode_4_or_more = viscode_counts[viscode_counts >= 4].index
filtered_mri = mri[mri['RID'].isin(rids_with_viscode_4_or_more)]

In [12]:
desired_viscodes = ['sc', 'm06', 'm12', 'm24']
filtered_mri = filtered_mri[filtered_mri['VISCODE2'].isin(desired_viscodes)]

filtered_mri = filtered_mri.groupby('RID').filter(
    lambda x: set(x['VISCODE2']) == set(desired_viscodes))

desired_viscodes = ['sc', 'm06', 'm12', 'm24']
filtered_mri = filtered_mri[filtered_mri['VISCODE2'].isin(desired_viscodes)]
filtered_mri = filtered_mri.groupby('RID').filter(
    lambda x: set(x['VISCODE2']) == set(desired_viscodes))

In [13]:
filtered_mri = filtered_mri.dropna(axis=1, how='all')

In [14]:
filtered_mri['VISCODE2'] = filtered_mri['VISCODE2'].replace('sc', 'bl')
filtered_mri.rename(columns={'VISCODE2': 'VISCODE'}, inplace=True)

In [15]:
min_count = 522
mri_all = filtered_mri.groupby('VISCODE').apply(lambda x: x.sample(n=min_count, random_state=1)).reset_index(drop=True).sort_values(by=['RID', 'VISCODE'])

C:\Users\anisr\AppData\Local\Temp\ipykernel_16836\2918354769.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mri_all = filtered_mri.groupby('VISCODE').apply(lambda x: x.sample(n=min_count, random_state=1)).reset_index(drop=True).sort_values(by=['RID', 'VISCODE'])


In [16]:
cols_to_keep_merge = df.columns.to_list() 

In [17]:
df_merge = pd.merge(df,mri_all, on = ['RID','VISCODE'])

In [18]:
df_sel = df_merge[mri_sel_lt]

In [19]:
df_merge 

,RID,VISCODE,DX,DX_bl,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,...,RightTemporalPole,RightThalamus,RightTransverseTemporal,RightTransverseTemporal,RightUnknown,RightUnknown,RightVentralDC,RightVessel,ThirdVentricle,WMHypoIntensities
0,3,bl,Dementia,AD,Male,18,Not Hisp/Latino,White,Married,1.000,...,"1,341.000","7,052.000",937.000,937.000,"6,286.000","6,286.000","3,915.000",228.000,"3,155.000","45,666.000"
1,3,m06,Dementia,AD,Male,18,Not Hisp/Latino,White,Married,1.000,...,"1,159.000","6,771.000",871.000,871.000,"5,978.000","5,978.000","3,951.000",220.000,"3,036.000","44,708.000"
2,3,m12,Dementia,AD,Male,18,Not Hisp/Latino,White,Married,1.000,...,"1,350.000","6,740.000",926.000,926.000,"6,151.000","6,151.000","3,900.000",233.000,"2,981.000","46,429.000"
3,3,m24,Dementia,AD,Male,18,Not Hisp/Latino,White,Married,1.000,...,"1,279.000","6,790.000",840.000,840.000,"5,149.000","5,149.000","3,754.000",203.000,"3,252.000","49,402.000"
4,5,bl,CN,CN,Male,16,Not Hisp/Latino,White,Married,0.000,...,"1,999.000","6,699.000","1,022.000","1,022.000","7,207.000","7,207.000","3,514.000",70.000,"1,548.000","5,781.000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,1352,m12,MCI,LMCI,Male,16,Not Hisp/Latino,White,Married,0.000,...,"1,471.000","6,093.000",497.000,497.000,"6,720.000","6,720.000","3,306.000",62.000,"2,319.000","4,339.000"
2084,692,bl,CN,CN,Female,19,Not Hisp/Latino,White,Widowed,1.000,...,"1,824.000","5,922.000",655.000,655.000,"5,983.000","5,983.000","3,612.000",27.000,"3,316.000","3,918.000"
2085,692,m06,CN,CN,Female,19,Not Hisp/Latino,White,Widowed,1.000,...,"1,546.000","5,909.000",544.000,544.000,"6,218.000","6,218.000","3,044.000",44.000,"3,143.000","5,040.000"
2086,692,m12,CN,CN,Female,19,Not Hisp/Latino,White,Widowed,1.000,...,"1,768.000","6,421.000",753.000,753.000,"6,382.000","6,382.000","3,344.000",45.000,"3,308.000","4,777.000"


In [20]:
df_sel['Icv'] = df_sel['Icv'].fillna(df_merge[df_merge['DX'] == 'CN']['Icv'].mean())

C:\Users\anisr\AppData\Local\Temp\ipykernel_16836\749858648.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['Icv'] = df_sel['Icv'].fillna(df_merge[df_merge['DX'] == 'CN']['Icv'].mean())


In [21]:
df_mri_sel_allnorm0 = df_sel.div(df_sel['Icv'], axis=0).drop('Icv', axis=1)

In [22]:
df_mri_sel_ICVscaled0 = df_mri_sel_allnorm0 * df_merge[df_merge['DX'] == 'CN']['Icv'].mean()

In [23]:
df_scaled_merge = pd.concat([df_mri_sel_ICVscaled0, df_merge[cols_to_keep_merge]], axis=1)

In [24]:
df_scaled_merge = df_scaled_merge.loc[:, ~df_scaled_merge.columns.duplicated()]

# Adding Sleep Medical History

In [25]:
import nltk

import re                            
import string                              

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer      
from nltk.tokenize import TweetTokenizer  

In [26]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anisr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
medhist = pd.read_csv("Imp_Files/RECMHIST_25Aug2023.csv")

In [28]:
medhist.loc[medhist['MHDESC'].str.contains("sleep \w{1,}/", case=False)][['RID', 'VISCODE2', 'MHDESC']]

,RID,VISCODE2,MHDESC
7672,1200,sc,SLEEP DISTURBANCE/INSOMNIA - 10/06
13691,298,m60,difficulty falling asleep unk/unk/1996
20127,4168,sc,Insomnia - Difficulty falling asleep r/t arthr...


In [29]:
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                           reduce_len=True)
stopwords_english = stopwords.words('english')
stemmer = PorterStemmer()

In [30]:
sleep_patterns = {
    'Sleep_Apnea': [
        r'\bsleep apnea\b', r'\bOSA\b', r'\bCPAP\b',
        r'\bobstructive sleep apnea\b', r'\bobstructive\b'
    ],
    'Restless_Legs': [
        r'\bRLS\b', r'\brestless legs\b', r'\brestless leg syndrome\b',
        r'\bleg movements\b', r'\brest less\b'
    ],
    'Insomnia': [
        r'\binsomnia\b', r'\bsleep difficulty\b', r'\bsleep onset\b',
        r'\bsleep maintenance\b', r'\bpoor sleep\b', r'\btrouble sleeping\b'
    ],
    'Sleep_Disturbance_Other': [
        r'\bsleep disturbance\b', r'\bhypersomnia\b', r'\bsomnambulism\b',
        r'\bparasomnia\b', r'\bnocturnal\b', r'\bdyssomnia\b', r'\bsleep disorder\b'
    ]
}

In [31]:
sleep_matches = []

for i in range(len(medhist['MHDESC'])):
    text = medhist['MHDESC'][i]

    tokens = tokenizer.tokenize(text)
    tokens_clean = [
        word for word in tokens
        if word.lower() not in stopwords_english and word not in string.punctuation
    ]
    stems = [stemmer.stem(word.lower()) for word in tokens_clean]
    stemmed_text = ' '.join(stems)

    for label, patterns in sleep_patterns.items():
        for pattern in patterns:
            if re.search(pattern, stemmed_text):
                sleep_matches.append({
                    'RID': medhist['RID'][i],
                    'VISCODE2': medhist['VISCODE2'][i],
                    'RECNO': medhist['RECNO'][i],
                    'MHNUM': medhist['MHNUM'][i],
                    'Matched_Pattern': pattern,
                    'Sleep_Label': label,
                    'Stemmed_Text': stemmed_text
                })
                break 

In [32]:
sleephist_ = pd.DataFrame(sleep_matches)
sleephist_[sleephist_['RID'].isin(df_merge['RID'])]['VISCODE2'].value_counts()

VISCODE2
sc     43
m60    33
m48    25
m72    18
m36     1
m84     1
Name: count, dtype: int64

In [33]:
sleephist_RIDs = sleephist_[sleephist_['RID'].isin(df_merge['RID'])]

In [34]:
sleephist_RIDs['Sleep_Label'].value_counts()

Sleep_Label
Sleep_Apnea    71
Insomnia       50
Name: count, dtype: int64

In [35]:
sleephist_RIDs.sort_values(by=['RID']).drop_duplicates(subset = ['RID'])['Sleep_Label'].value_counts()

Sleep_Label
Sleep_Apnea    35
Insomnia       28
Name: count, dtype: int64

In [36]:
sleephist_RIDs[sleephist_RIDs['Sleep_Label'] == 'Insomnia'].drop_duplicates(subset=['RID'])['RID'].unique(
).shape, sleephist_RIDs[sleephist_RIDs['Sleep_Label'] == 'Sleep_Apnea'].drop_duplicates(subset=['RID'])['RID'].unique().shape,

((31,), (35,))

In [37]:
df_sleep_flags = sleephist_RIDs[['RID', 'Sleep_Label']].copy()
df_sleep_flags['Flag'] = 1

df_sleep_wide = df_sleep_flags.pivot_table(
    index='RID',
    columns='Sleep_Label',
    values='Flag',
    aggfunc='max',
    fill_value=0
).reset_index()

for col in ['Sleep_Apnea', 'Insomnia', 'Restless_Legs', 'Sleep_Disturbance_Other']:
    if col not in df_sleep_wide.columns:
        df_sleep_wide[col] = 0
df_sleep_wide = df_sleep_wide[['RID', 'Sleep_Apnea', 'Insomnia']]

In [38]:
all_rids = df_merge[['RID']].drop_duplicates()

df_sleep_flags = sleephist_RIDs[['RID', 'Sleep_Label']].copy()
df_sleep_flags['Flag'] = 1
df_sleep_wide = df_sleep_flags.pivot_table(
    index='RID',
    columns='Sleep_Label',
    values='Flag',
    aggfunc='max',
    fill_value=0
).reset_index()

sleep_status = all_rids.merge(df_sleep_wide, on='RID', how='left')
sleep_status = sleep_status.fillna(0)
sleep_status[['Sleep_Apnea', 'Insomnia']] = sleep_status[[
    'Sleep_Apnea', 'Insomnia']].astype(int)

In [39]:
sleep_status['Insomnia'].value_counts(
), sleep_status['Sleep_Apnea'].value_counts()

(Insomnia
 0    491
 1     31
 Name: count, dtype: int64,
 Sleep_Apnea
 0    487
 1     35
 Name: count, dtype: int64)

In [40]:
df_scaled_merge_sleep_hist = df_scaled_merge.merge(sleep_status, on='RID', how='left')

# Adding Medical History 

In [41]:
medical_his = pd.read_csv("Files/MEDHIST_19Feb2025.csv")
medical_his = medical_his[['RID', 'VISCODE2','MHPSYCH','MH2NEURL','MH4CARD','MH5RESP']]

In [42]:
medi_his_sc = medical_his[(medical_his['RID'].isin(df_scaled_merge_sleep_hist['RID'])) & (medical_his['VISCODE2'] == 'sc')]

In [43]:
df_scaled_merge_hist = df_scaled_merge_sleep_hist.merge(
    medi_his_sc, on=['RID'], how='left')

In [44]:
df_scaled_merge_hist['MH'] = df_scaled_merge_hist[['MHPSYCH', 'MH2NEURL', 'MH4CARD', 'MH5RESP']].sum(axis=1)
df_scaled_merge_hist.drop(columns=['MHPSYCH', 'MH2NEURL', 'MH4CARD', 'MH5RESP'], inplace=True) 

# Adding Longitudinal Sleep vars

In [45]:
sleep_sev = pd.read_csv("Files/All_Subjects_NPIQ_07Feb2025.csv")[['RID', 'VISCODE2','NPIK','NPIKSEV']]

In [46]:
sleep_sev_part = sleep_sev[(sleep_sev['RID'].isin(df_scaled_merge_hist['RID'])) & (sleep_sev['VISCODE2'].isin(['bl', 'm06','m12','m24']))]

In [47]:
sleep_sev_part.sort_values(by=['RID', 'VISCODE2'],inplace=True)

C:\Users\anisr\AppData\Local\Temp\ipykernel_16836\707089745.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep_sev_part.sort_values(by=['RID', 'VISCODE2'],inplace=True)


In [48]:
sleep_sev_part['NPIK'].value_counts(), sleep_sev_part['NPIKSEV'].value_counts()

(NPIK
 0.000     1732
 1.000      332
 2.000       23
 -1.000       1
 Name: count, dtype: int64,
 NPIKSEV
 -4.000    1756
 1.000      217
 2.000       94
 3.000       21
 Name: count, dtype: int64)

In [49]:
sleep_params_npik = {2: 0, -1:0}
sleep_sev_part['NPIK'] = sleep_sev_part['NPIK'].replace(sleep_params_npik) 
sleep_sev_part['NPIK'].value_counts()

C:\Users\anisr\AppData\Local\Temp\ipykernel_16836\1316152649.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep_sev_part['NPIK'] = sleep_sev_part['NPIK'].replace(sleep_params_npik)


NPIK
0.000    1756
1.000     332
Name: count, dtype: int64

In [50]:
sleep_params_npiksev = {-4:0} 
sleep_sev_part['NPIKSEV'] = sleep_sev_part['NPIKSEV'].replace(sleep_params_npiksev)
sleep_sev_part['NPIKSEV'].value_counts()

C:\Users\anisr\AppData\Local\Temp\ipykernel_16836\3659750778.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep_sev_part['NPIKSEV'] = sleep_sev_part['NPIKSEV'].replace(sleep_params_npiksev)


NPIKSEV
0.000    1756
1.000     217
2.000      94
3.000      21
Name: count, dtype: int64

In [51]:
sleep_sev_part.rename(columns={"VISCODE2": "VISCODE"}, inplace=True)
df_merged_ = df_scaled_merge_hist.merge(sleep_sev_part, on=['RID', 'VISCODE'])

C:\Users\anisr\AppData\Local\Temp\ipykernel_16836\191701481.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep_sev_part.rename(columns={"VISCODE2": "VISCODE"}, inplace=True)


# Adding BMI

In [52]:
vitals = pd.read_csv("Files/All_Subjects_VITALS_19Feb2025.csv")

In [53]:
unique_rids = vitals['RID'].unique()
filtered_data = vitals[(vitals['VSWEIGHT'] > 0) & (vitals['VSHEIGHT'] > 0)]
viscode2_per_rid = filtered_data.groupby(
    'RID')['VISCODE2'].first().reset_index()

def calculate_bmi(weight, weight_unit, height, height_unit):
    if weight_unit == 2:
        weight = weight * 0.453592
    if height_unit == 1:
        height = height * 0.0254
    if height > 0:
        return weight / (height ** 2)
    return None


valid_data = vitals[(vitals['VSWEIGHT'] > 0) & (vitals['VSHEIGHT'] > 0)]

valid_data['BMI'] = valid_data.apply(lambda row: calculate_bmi(
    row['VSWEIGHT'], row['VSWTUNIT'], row['VSHEIGHT'], row['VSHTUNIT']), axis=1)
bmi_per_rid = valid_data.groupby('RID').apply(
    lambda group: group.loc[group.index[0], ['RID', 'BMI']]).reset_index(drop=True)

C:\Users\anisr\AppData\Local\Temp\ipykernel_16836\1473836708.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['BMI'] = valid_data.apply(lambda row: calculate_bmi(
C:\Users\anisr\AppData\Local\Temp\ipykernel_16836\1473836708.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bmi_per_rid = valid_data.groupby('RID').apply(


In [54]:
first_valid_rows = valid_data.sort_values(
    'VISCODE2').groupby('RID').first().reset_index()
bmi_per_rid = first_valid_rows[['RID', 'BMI']]

In [55]:
bmi_per_rid[bmi_per_rid['RID'].isin(df_merged_['RID'])]

,RID,BMI
1,3,10.928
3,5,12.273
8,10,11.672
11,14,59.157
13,16,11.018
...,...,...
1153,1414,70.264
1155,1418,0.005
1156,1419,65.444
1160,1425,0.005


In [56]:
df_merged_bmi = df_merged_.merge(bmi_per_rid, on='RID')

# Adding Psychometric tests

In [57]:
psychom = pd.read_csv("Files/UWNPSYCHSUM_13Feb2025.csv")
psychom = psychom[['RID', 'VISCODE2', 'ADNI_MEM',
                   'ADNI_EF', 'ADNI_VS', 'ADNI_LAN', 'ADNI_EF2']]
psychom = psychom.rename(columns={'VISCODE2': 'VISCODE'})

In [58]:
psychom[psychom['RID'].isin(df_merged_bmi['RID'])]['RID'].nunique()

522

In [59]:
merged_file = pd.merge(df_merged_bmi, psychom, on=['RID', 'VISCODE'])
merged_file

,RightEntorhinal,RightTemporalPole,RightParahippocampal,RightInferiorTemporal,RightMiddleTemporal,RightFusiform,RightInferiorParietal,RightIsthmusCingulate,RightBankssts,RightPrecuneus,...,VISCODE2,MH,NPIK,NPIKSEV,BMI,ADNI_MEM,ADNI_EF,ADNI_VS,ADNI_LAN,ADNI_EF2
0,791.847,"1,078.037","1,512.146","6,716.630","8,350.166","6,435.264","9,580.142","1,771.808","2,320.875","6,521.281",...,sc,1,0.000,0.000,10.928,-1.021,-1.144,-0.880,-1.368,-1.062
1,958.134,938.696,"1,614.978","6,975.020","7,541.153","5,677.530","9,678.528","1,755.904","2,343.095","6,551.432",...,sc,1,0.000,0.000,10.928,-1.187,-0.674,-1.221,-1.638,-0.670
2,676.398,"1,094.889","1,440.388","6,116.781","7,896.178","6,183.285","9,557.977","1,623.680","2,230.330","6,147.600",...,sc,1,0.000,0.000,10.928,-0.914,-0.969,-0.548,-1.837,-1.034
3,442.104,"1,037.524","1,379.850","6,111.577","7,557.134","5,892.553","9,046.496","1,727.044","2,247.022","6,243.802",...,sc,1,0.000,0.000,10.928,-1.369,-1.511,0.739,-1.700,-1.605
4,"2,376.161","1,881.167","2,169.130","9,288.203","10,896.465","12,424.737","13,952.067","1,972.449","2,349.812","8,757.449",...,sc,0,0.000,0.000,12.273,0.743,0.165,0.739,0.259,-0.152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,"3,607.921","1,341.909","1,940.341","5,798.216","8,825.950","8,448.282","8,627.994","1,805.329","1,208.722","7,881.779",...,sc,2,0.000,0.000,0.001,0.713,0.173,0.739,0.402,0.286
2084,"2,656.701","1,875.318","2,553.888","6,926.546","9,415.661","8,203.490","9,889.631","2,043.933","1,624.453","7,987.581",...,sc,0,0.000,0.000,42.072,0.766,0.490,-0.080,0.485,0.475
2085,"2,380.565","1,591.160","2,251.914","6,975.992","8,889.296","8,375.719","8,921.201","1,996.669","1,708.490","7,886.844",...,sc,0,0.000,0.000,42.072,0.758,0.564,-0.858,0.755,0.327
2086,"2,150.860","1,809.957","2,368.915","6,506.837","9,778.683","8,074.170","10,269.051","2,052.582","2,153.931","8,445.785",...,sc,0,0.000,0.000,42.072,1.249,0.473,-1.369,0.303,0.294


# Other preprocessing

In [60]:
merged_file.drop(columns=['VISCODE2'], inplace=True, errors='ignore')

In [63]:
merged_file.loc[merged_file.isna().any(axis=1)][['RID','VISCODE','DX','ADNI_MEM','ADNI_EF','ADNI_VS','ADNI_LAN','ADNI_EF2']]

,RID,VISCODE,DX,ADNI_MEM,ADNI_EF,ADNI_VS,ADNI_LAN,ADNI_EF2
827,682,m24,Dementia,-2.699,NaN,-2.013,-1.541,-1.592
1303,1157,m24,Dementia,NaN,NaN,NaN,NaN,NaN
1843,1014,m24,CN,0.331,NaN,0.672,0.772,NaN
1844,1014,m24,CN,0.331,NaN,0.672,0.772,NaN
1891,1205,m24,NaN,NaN,NaN,NaN,NaN,NaN
1933,1382,m24,Dementia,-3.166,NaN,-2.427,-3.337,-1.592


In [65]:
merged_file.loc[merged_file.isna().any(axis=1)]['RID'].to_list()

[682, 1157, 1014, 1014, 1205, 1382]

In [68]:
merged_file[merged_file['RID'].isin(merged_file.loc[merged_file.isna().any(axis=1)]['RID'].to_list())][['RID','VISCODE','DX','ADNI_MEM','ADNI_EF','ADNI_VS','ADNI_LAN','ADNI_EF2']].sort_values(by=['RID','VISCODE'])

,RID,VISCODE,DX,ADNI_MEM,ADNI_EF,ADNI_VS,ADNI_LAN,ADNI_EF2
824,682,bl,Dementia,-1.848,-1.227,-0.080,-0.613,-1.343
825,682,m06,Dementia,-1.176,-0.827,-0.656,-0.897,-0.920
826,682,m12,Dementia,-1.887,-1.343,-1.146,-1.416,-1.251
827,682,m24,Dementia,-2.699,NaN,-2.013,-1.541,-1.592
1176,1014,bl,CN,0.745,-0.603,-0.080,-0.106,-0.551
1177,1014,m06,CN,1.114,-0.056,0.739,0.114,-0.018
1178,1014,m12,CN,0.710,-0.206,0.739,-0.139,-0.359
1843,1014,m24,CN,0.331,NaN,0.672,0.772,NaN
1844,1014,m24,CN,0.331,NaN,0.672,0.772,NaN
1300,1157,bl,Dementia,-0.987,-1.283,-0.739,-1.181,-1.467


In [70]:
merged_file.sort_values(by=['RID','VISCODE'],inplace=True)

In [74]:
merged_file.fillna(method='ffill').loc[merged_file['RID'].isin([682, 1157, 1014, 1014, 1205, 1382])][['RID','VISCODE','DX','ADNI_MEM','ADNI_EF','ADNI_VS','ADNI_LAN','ADNI_EF2']]

C:\Users\anisr\AppData\Local\Temp\ipykernel_16836\3152241661.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_file.fillna(method='ffill').loc[merged_file['RID'].isin([682, 1157, 1014, 1014, 1205, 1382])][['RID','VISCODE','DX','ADNI_MEM','ADNI_EF','ADNI_VS','ADNI_LAN','ADNI_EF2']]


,RID,VISCODE,DX,ADNI_MEM,ADNI_EF,ADNI_VS,ADNI_LAN,ADNI_EF2
824,682,bl,Dementia,-1.848,-1.227,-0.080,-0.613,-1.343
825,682,m06,Dementia,-1.176,-0.827,-0.656,-0.897,-0.920
826,682,m12,Dementia,-1.887,-1.343,-1.146,-1.416,-1.251
827,682,m24,Dementia,-2.699,-1.343,-2.013,-1.541,-1.592
1176,1014,bl,CN,0.745,-0.603,-0.080,-0.106,-0.551
1177,1014,m06,CN,1.114,-0.056,0.739,0.114,-0.018
1178,1014,m12,CN,0.710,-0.206,0.739,-0.139,-0.359
1843,1014,m24,CN,0.331,-0.206,0.672,0.772,-0.359
1844,1014,m24,CN,0.331,-0.206,0.672,0.772,-0.359
1300,1157,bl,Dementia,-0.987,-1.283,-0.739,-1.181,-1.467


In [75]:
merged_file.fillna(method='ffill',inplace=True)

C:\Users\anisr\AppData\Local\Temp\ipykernel_16836\2147167002.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_file.fillna(method='ffill',inplace=True)


In [76]:
merged_file.to_csv("Latest_all_in_one.csv",index=False)